In [1]:
!pip install torchmetrics
!pip install comet_ml torch torchvision tqdm

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: /usr/bin/python -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: /usr/bin/python -m pip install --upgrade pip


In [2]:
import sys
 
sys.path.insert(0, "../")

from EnsembleFramework import Framework

In [3]:
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.utils import add_self_loops

dataset_name = 'Cora'
split = "public"
dataset = Planetoid(root='/tmp/Cora', name=dataset_name, split=split)
dataset.transform = T.NormalizeFeatures()

features =  dataset[0].x
y =  dataset[0].y

test =  dataset[0].test_mask
train = dataset[0].train_mask
val =  dataset[0].val_mask

edge_index = dataset[0].edge_index 
edge_index = add_self_loops(edge_index)[0]

In [4]:
hops_to_best_configs = {0: {'inter_layer_normalize': True,
  'use_pseudo_attention': True,
  'cosine_eps': 0.005,
  'dropout_attn': 0.0},
 1: {'inter_layer_normalize': False,
  'use_pseudo_attention': True,
  'cosine_eps': 0.005,
  'dropout_attn': 0.0},
 2: {'inter_layer_normalize': False,
  'use_pseudo_attention': True,
  'cosine_eps': 0.005,
  'dropout_attn': 0.00011111111111111112},
 3: {'inter_layer_normalize': False,
  'use_pseudo_attention': True,
  'cosine_eps': 0.005,
  'dropout_attn': 0.00022222222222222223},
 4: {'inter_layer_normalize': False,
  'use_pseudo_attention': True,
  'cosine_eps': 0.005,
  'dropout_attn': 0.0},
 5: {'inter_layer_normalize': False,
  'use_pseudo_attention': True,
  'cosine_eps': 0.005,
  'dropout_attn': 0.0},
 6: {'inter_layer_normalize': False,
  'use_pseudo_attention': True,
  'cosine_eps': 0.005,
  'dropout_attn': 0.0003333333333333334},
 7: {'inter_layer_normalize': False,
  'use_pseudo_attention': True,
  'cosine_eps': 0.005,
  'dropout_attn': 0.0},
 8: {'inter_layer_normalize': False,
  'use_pseudo_attention': True,
  'cosine_eps': 0.06,
  'dropout_attn': 0.00011111111111111112},
 9: {'inter_layer_normalize': False,
  'use_pseudo_attention': True,
  'cosine_eps': 0.06,
  'dropout_attn': 0.00022222222222222223},
 10: {'inter_layer_normalize': False,
  'use_pseudo_attention': True,
  'cosine_eps': 0.06,
  'dropout_attn': 0.00011111111111111112},
 15: {'inter_layer_normalize': True,
  'use_pseudo_attention': True,
  'cosine_eps': 0.005,
  'dropout_attn': 0.0},
 20: {'inter_layer_normalize': True,
  'use_pseudo_attention': True,
  'cosine_eps': 0.005,
  'dropout_attn': 0.0},
 30: {'inter_layer_normalize': False,
  'use_pseudo_attention': True,
  'cosine_eps': 0.06,
  'dropout_attn': 0.00044444444444444447},
 40: {'inter_layer_normalize': False,
  'use_pseudo_attention': True,
  'cosine_eps': 0.06,
  'dropout_attn': 0.0003333333333333334},
 50: {'inter_layer_normalize': False,
  'use_pseudo_attention': True,
  'cosine_eps': 0.06,
  'dropout_attn': 0.0}}

In [5]:
from torch.nn.functional import normalize
def user_function(kwargs):
    return  normalize(kwargs["original_features"] + kwargs["summed_neighbors"], p=2.0, dim = 1)
    
hops_list = [1,3,4,6,7,8,9,15, 20, 30, 40, 50]#[0,2,5,10, 15, 20, 30]
framework = Framework([user_function for i in hops_list], 
                     hops_list=hops_list, ## to obtain best for local neighborhood
                     clfs=[],
                     gpu_idx=0,
                     handle_nan=0.0,
                    attention_configs=[hops_to_best_configs[hop] for hop in hops_list])
new_features_list = framework.get_features(features, edge_index, None)

/home/dwalke/git/graph_aware_ml/logistic_regression/../EnsembleFramework.py:146: UserWarning: The reduce argument of torch.scatter with Tensor src is deprecated and will be removed in a future PyTorch release. Use torch.scatter_reduce instead for more reduction options. (Triggered internally at ../aten/src/ATen/native/TensorAdvancedIndexing.cpp:230.)
  summed_exp_score = torch.zeros_like(exp_score).scatter(0, target,exp_score, reduce="add")


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import ParameterGrid
from tqdm.notebook import tqdm
from hyperopt import fmin, tpe, hp,STATUS_OK

penalty = ["l1","l2", None, "elasticnet"]
max_iter = [2**i for i in range(6, 15)]

lr_choices = {
    'penalty': penalty,
    'max_iter': max_iter,
    
}

space = {
    **{key: hp.choice(key, value) for key, value in lr_choices.items()},
    'tol': hp.loguniform('tol', -11, -3),
    'C': hp.uniform('C', 0.0, 10),
    'l1_ratio': hp.uniform('l1_ratio', 0.0, 1.0)
}

clf = LogisticRegression
# hops_to_params=dict({})
# hops_to_score=dict({})

for i, hops in tqdm(enumerate(hops_list)):
    new_features = new_features_list[i].cpu()
    
    def objective(params):
        model = clf(**params, random_state = 42, solver= "saga")
        model.fit(new_features[train], y[train])
        y_pred = model.predict(new_features[val])
        score = accuracy_score(y[val], y_pred)
        return {'loss': -score, 'status': STATUS_OK}
    
    best_params = fmin(objective, space, algo=tpe.suggest, max_evals=1_000)

    for key,value in lr_choices.items():
        best_params[key] = value[best_params[key]]

    best_model = clf(**best_params, random_state = 42, solver= "saga")
    best_model.fit(new_features[train], y[train])
    y_pred = best_model.predict(new_features[val])
    
    hops_to_params[hops] = best_params
    hops_to_score[hops] = accuracy_score(y[val], y_pred)

0it [00:00, ?it/s]


  0%|                                                                          | 0/1000 [00:00<?, ?trial/s, best loss=?]

/home/dwalke/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1172: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=None)
  warnings.warn(

/home/dwalke/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(




  0%|                                                            | 1/1000 [00:00<05:13,  3.19trial/s, best loss: -0.666]

/home/dwalke/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1172: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(



...

/home/dwalke/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1172: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(




 65%|█████████████████████████████████████▉                    | 654/1000 [04:18<01:16,  4.51trial/s, best loss: -0.678]

/home/dwalke/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1172: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(

/home/dwalke/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1172: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(




 66%|██████████████████████████████████████                    | 656/1000 [04:18<01:06,  5.18trial/s, best loss: -0.678]

/home/dwalke/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1172: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(

/home/dwalke/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1172: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=None)
  warnings.warn(

/home/dwalke/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(



In [ ]:
hops_to_params

In [ ]:
import pandas as pd
df = pd.DataFrame(columns=["best_params"], index=hops_to_params.keys())
df["best_params"] = hops_to_params.values()
df.to_csv("normalized_origin_features_plus_summed_neighbors.csv")
df